In [1]:
#!pip install faiss-gpu
#!pip install openpyxl
#!pip install --upgrade ollama
#!pip install openpyxl

In [2]:
from ollama import chat
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_ollama import OllamaEmbeddings
from langchain_ollama import ChatOllama
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, ValidationError
import pandas as pd
from pandas import json_normalize
import json
import os
import glob
import re
from typing import Optional, Literal
import collections

In [3]:
loader = PDFPlumberLoader("docs/urgencias/Hist6.pdf")
docs = loader.load()

# Check the number of pages
print("Number of pages in the PDF:",len(docs))

Number of pages in the PDF: 2


In [4]:
class InformacionPaciente(BaseModel):
    id_episodio: str = Field(
        description="Clave identificadora del episodio o historia clínica (NºHª)"
    )
    fecha_nacimiento: str = Field(
        description="Fecha de nacimiento del paciente"
    )
    edad: int = Field(
        description="Edad del paciente"
    )
    sexo: str = Field(
        description="Sexo del paciente"
    )
    fecha_ingreso_hospital: str = Field(
        description="Fecha de ingreso en el hospital"
    )
    fecha_ingreso_uci: str = Field(
        description="Fecha de ingreso en la UCI"
    )
    hora_ingreso_uci: str = Field(
        description="Hora de ingreso en la UCI"
    )
    estancia_preuci: int = Field(
        description="Número de días en los que ha estado hospitalizado antes de entrar en la UCI"
    )
    reingreso: bool = Field(
        description="¿Ha sido ingresado el paciente con anterioridad?"
    )
    numero_ingresos_previos: int = Field(
        description="Número de veces que el paciente ha sido ingresado con anterioridad"
    )
    tipo_ingreso_uci: str = Field(
        description="¿Ha sido el ingreso en UCI del paciente de tipo programado o de tipo urgente?"
    )
    tipo_paciente: str = Field(
        description="¿Se trata de un paciente de tipo médico o de tipo quirúrgico?"
    )
    procedencia: str = Field(
        description="Indíquese si el origen del paciente ha sido el quirófano, una hospitalización convencional, la sala de urgencias u otro área hospitalaria"
    )
    fecha_alta_uci: str = Field(
        description="Fecha de alta de la UCI"
    )
    hora_alta_uci: str = Field(
        description="Hora de alta de la UCI"
    )
    estancia_uci: int = Field(
        description="Número de días que el paciente ha estado en la UCI"
    )
    estancia_postuci: str = Field(
        description="Número de días que el paciente ha estado hospitalizado después de salir de la UCI"
    )
    destino_alta_uci: str = Field(
        description="Destino del paciente tras haber sido dado de alta en la UCI: exitus letalis, envío a domicilio, traslado a otro centro hospitalario u hospitalización convencional"
    )
    estado_paciente: str = Field(
        description="Estado del paciente tras haber sido dado de alta del hospital: vivo o fallecido"
    )
    motivo_ingreso: str = Field(
        description="Motivo de ingreso en la UCI"
    )
    diagnostico: str = Field(
        description="Diagnóstico principal de acuerdo con la CIE (indíquese la enfermedad con su código, como por ejemplo aquí: D42 - Neoplasia de comportamiento incierto de meninges)"
    )

In [5]:
InformacionPaciente.update_forward_refs()

/tmp/ipykernel_3043495/2475997593.py:1: PydanticDeprecatedSince20: The `update_forward_refs` method is deprecated; use `model_rebuild` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  InformacionPaciente.update_forward_refs()


In [6]:
llm_to_use ="mistral"

In [7]:
text_splitter = SemanticChunker(OllamaEmbeddings(model=llm_to_use, base_url="http://127.0.0.1:11434"))
documents = text_splitter.split_documents(docs)

In [8]:
# Paso 1: Configuración del parser para la salida estructurada con Pydantic
parser = PydanticOutputParser(pydantic_object=InformacionPaciente)

# Paso 2: Configuración del embedder y el vector retriever
embedder = OllamaEmbeddings(model=llm_to_use, base_url="http://127.0.0.1:11434")
vector = FAISS.from_documents(documents, embedder)
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# Paso 3: Configuración del modelo LLM
llm = Ollama(model=llm_to_use, temperature=0)


/tmp/ipykernel_3043495/4031420779.py:10: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model=llm_to_use, temperature=0)


In [9]:
consulta = "Obtener la información del paciente"
# Recuperar documentos relevantes
documentos_relevantes = retriever.get_relevant_documents(consulta)
if not documentos_relevantes:
    print("No se encontraron documentos relevantes para la consulta.")

print()

# Formatear el contexto con los documentos recuperados
historia_clinica = "\n\n".join([doc.page_content for doc in documentos_relevantes if hasattr(doc, 'page_content') and doc.page_content])
if not historia_clinica:
    print("Historia clínica vacía después de procesar los documentos.")

print()

# Generar el esquema JSON
formato_resultado = InformacionPaciente.model_json_schema()
#print("Esquema JSON generado:", formato_resultado)

print()

# Crear los mensajes con el contexto
response = chat(
    messages=[
        {
            'role': 'user',
            'content': (
                "Extráigase la información necesaria de la historia clínica: " f"{historia_clinica}\n\n"
            ),
        }
    ],
    model=llm_to_use,
    format=formato_resultado
)


/tmp/ipykernel_3043495/604880271.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  documentos_relevantes = retriever.get_relevant_documents(consulta)


In [10]:
try:
    # Convierte el contenido del modelo a una cadena JSON
    resultado = InformacionPaciente.model_validate_json(response.message.content).json()
    #print(resultado)
except ValidationError as e:
    print(f"Errores encontrados: {e}")
except json.JSONDecodeError as e:
    print(f"Error al decodificar JSON: {e}")

/tmp/ipykernel_3043495/4222015083.py:3: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  resultado = InformacionPaciente.model_validate_json(response.message.content).json()


In [11]:
file_path = "docs/urgencias/ResultadosMistral.xlsx"
df_existente = pd.read_excel(file_path)

data = json.loads(resultado)
flattened_data = json_normalize(data)
df_nuevo = pd.DataFrame(flattened_data)
pd.set_option('display.max_columns', None)
df_actualizado = pd.concat([df_existente, df_nuevo], ignore_index=True)
df_actualizado.to_excel(file_path, index=False)

In [12]:
df_actualizado

,id_episodio,fecha_nacimiento,edad,sexo,fecha_ingreso_hospital,fecha_ingreso_uci,hora_ingreso_uci,estancia_preuci,reingreso,numero_ingresos_previos,tipo_ingreso_uci,tipo_paciente,procedencia,fecha_alta_uci,hora_alta_uci,estancia_uci,estancia_postuci,destino_alta_uci,estado_paciente,motivo_ingreso,diagnostico
0,One de los pacientes más complejos del servici...,12/11/2019,79,Masculino,28/11/19,12/11/2019,08:00,2,True,1,Trauma facial con fractura de mandíbula,Adulto,Quirófano de Cirugía General en el postoperato...,03/01/2020,08:00,2,Not specified,Día de liberación,Seis estados clínicos diferentes en un solo pa...,"Infección por P. aeruginosa, K. pneumoniae y E...","Sepsis, Reapertura de fístula biliar, Bloqueo ..."
1,ingreso al hospital,No proporcionada,68,femenino,22/04/2023,23/04/2023,No proporcionado,1,False,23456,Inferior a 14 días de hospitalización,Urgencia,Cirugía torácica en el postoperatorio de Lobec...,23/04/2023,No proporcionada,2,No proporcionado,PACIENTE: Antonia María González Rubio,Inferior a 14 días de hospitalización,Paciente de 68 años que ingresa procedente de ...,Adenocarcinoma acinar bien diferenciado. TTF1 ...
2,https://fdo.com/episodio/2023-06-14/2023-06-15,No disponible,65,Mujer,2023-06-14,2023-06-14,08:00,23456,False,1,Postoperatorio de ByPass Gástrico,Cirugía gastrointestinal,Quirúfano,2023-06-15,08:00,1,Falta de información,Planta de hospitalización convencional,Estable,Postoperatorio inmediato de ByPass Gástrico,"ByPass Gástrico, Cirugía gastrointestinal, Int..."
3,076456873,NaN,67,Femenino,07/01/2023,07/01/2023,NaN,0,False,1,Posoperatorio de gastrectomía radical ampliada,Internado,Quirófano de Cirugía General,NaN,NaN,0,Planta de hospitalización convencional,Planta de hospitalización convencional,Estabilizado,"Dolor intenso a nivel de epigástrico, diagnóst...",Postoperatorio de gastrectomía radical ampliad...
4,0362346,NaN,61,M,27/02/2023,27/02/2023,NaN,1,False,1,Postoperatorio de resección de meningioma,Adulto,Quirófano de Neurocirugía,28/02/2023,NaN,1,",",Planta de hospitalización convencional,Estable,NaN,Paciente de 61 años que ingresa procedente de ...
5,0325476868,N/A,0,M,20/04/2021,20/04/2021,N/A,0,False,0,Post intervención TAVI,Hospitalizado,Sala de hemodinámica,21/04/21,N/A,1,. . . (si aplica),Planta de hospitalización convencional,Favorable,Paciente de años que ingresa procedente sala d...,Implantación de TAVI + Stent en ADA. Anteceden...
